In [1]:
corpus_path = '../_data/preprocessed/2020_11_17_220802[text].txt'

class Documents:
    def __init__(self, path):
        self.path = path
    def __iter__(self):
        with open(self.path, encoding='utf-8') as f:
            for doc in f:
                yield doc.strip().split()

documents = Documents(corpus_path)

In [2]:
import pyLDAvis

In [3]:
import gensim

dictionary = gensim.corpora.Dictionary(documents)
print('dictionary size : %d' % len(dictionary))

D:\PF\_Programming\Anaconda\envs\Pytorch\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


dictionary size : 75424


In [4]:
from collections import Counter

min_count = 40
word_counter = Counter((word for words in documents for word in words))
removal_word_idxs = {
    dictionary.token2id[word] for word, count in word_counter.items()
    if count < min_count
}

dictionary.filter_tokens(removal_word_idxs)
dictionary.compactify()
print('dictionary size : %d' % len(dictionary))

dictionary size : 7436


In [5]:
class Corpus:
    def __init__(self, path, dictionary):
        self.path = path
        self.dictionary = dictionary
        self.length = 0
    def __iter__(self):
        with open(self.path, encoding='utf-8') as f:
            for doc in f:
                yield self.dictionary.doc2bow(doc.split())
    def __len__(self):
        if self.length == 0:
            with open(self.path, encoding='utf-8') as f:
                for i, doc in enumerate(f):
                    continue
            self.length = i + 1
        return self.length

corpus = Corpus(corpus_path, dictionary)
for i, doc in enumerate(corpus):
    if i >= 5: break
    print(doc)

[(0, 1876), (1, 598), (2, 139), (3, 60), (4, 140), (5, 1141), (6, 44), (7, 333), (8, 103), (9, 987), (10, 50), (11, 77), (12, 165), (13, 75), (14, 4052), (15, 1495), (16, 102), (17, 351), (18, 46), (19, 50), (20, 796), (21, 141), (22, 71), (23, 56), (24, 330), (25, 40), (26, 163), (27, 82), (28, 178), (29, 172), (30, 89), (31, 217), (32, 201), (33, 156), (34, 57), (35, 76), (36, 55), (37, 41), (38, 4060), (39, 47), (40, 110), (41, 65), (42, 146), (43, 667), (44, 44), (45, 271), (46, 52), (47, 45), (48, 747), (49, 2605), (50, 71), (51, 137), (52, 713), (53, 79), (54, 285), (55, 58), (56, 1289), (57, 63), (58, 114), (59, 470), (60, 47), (61, 97), (62, 46), (63, 217), (64, 56), (65, 48), (66, 46), (67, 177), (68, 548), (69, 54), (70, 55), (71, 3173), (72, 132), (73, 217), (74, 51), (75, 389), (76, 58), (77, 50), (78, 181), (79, 79), (80, 49), (81, 40), (82, 59), (83, 397), (84, 50), (85, 603), (86, 53), (87, 351), (88, 124), (89, 81), (90, 74), (91, 59), (92, 98), (93, 53), (94, 199), (95

In [6]:
import pickle

data_path = '../_model/2016-10-20-news-bow.pkl'

with open(data_path, 'rb') as f:
    params = pickle.load(f)
    x = params['x']
    index2word = params['index2word']
    word2index = params['word2index']

In [14]:
with open("../_model/2016-10-20-news-bow.pkl","rb") as fr:
    data = pickle.load(fr)
print(data)

{'x': <30091x9774 sparse matrix of type '<class 'numpy.int64'>'
	with 1934111 stored elements in Compressed Sparse Row format>, 'word2index': {'580': 606, '경제협력개발기구': 1238, '태국': 8752, '2000여': 280, '충돌': 8508, '불허': 4224, '쇼핑': 4973, '천하': 8235, '원내대책회의에서': 6349, '한양대': 9296, '극대화': 1825, '증명': 7897, '고교': 1287, '혜택': 9537, '파이': 8911, '오피스': 6166, '성격': 4742, '카메오': 8596, '현대상선': 9475, '신입생': 5340, '최대': 8382, '총탄': 8363, '티셔츠': 8890, '대상자': 2625, '장수': 7169, '리그': 3061, '배출': 3839, '의약품': 6627, '평상시': 9003, '230': 356, '지역민': 7961, '참가자': 8153, '위해서라': 6439, '감시': 907, '빅데이터': 4308, '대손비용': 2631, '때문': 2965, '스크린도어': 5122, '연면적': 5982, '저희': 7264, '여행사': 5928, '김정철': 2082, '공공': 1393, '대구': 2585, '임무': 6953, '자세': 7069, '김만복': 2036, '초과': 8310, '거장': 1054, '난사': 2214, '단계별': 2493, '저전력': 7251, '음악회': 6586, '호소': 9543, '술집': 5093, '실물': 5380, '사업장': 4378, '양승': 5735, '도둑': 2725, '세밀': 4810, '신한금융투자': 5359, '말들': 3181, '김상헌': 2048, '국방': 1690, '사실': 4367, '부활': 4143, '굳건': 1736, '김지훈'

In [7]:
import gensim
from gensim.corpora.dictionary import Dictionary

corpus = gensim.matutils.Sparse2Corpus(x, documents_columns=False)
dictionary = Dictionary.from_corpus(
    corpus,
    id2word = dict(enumerate(index2word))
)

In [8]:
from gensim.models import LdaModel

lda_model_path = '../_model/testLdaModel'

lda_model = LdaModel(corpus, id2word=dictionary, num_topics=50)
with open(lda_model_path, 'wb') as f:
    pickle.dump(lda_model, f)

In [9]:
def get_topic_term_prob(lda_model):
    topic_term_freqs = lda_model.state.get_lambda()
    topic_term_prob = topic_term_freqs / topic_term_freqs.sum(axis=1)[:, None]
    return topic_term_prob

In [10]:
print(lda_model.alpha.shape) # (n_topics,)
print(lda_model.alpha.sum()) # 1.0

topic_term_prob = get_topic_term_prob(lda_model)
print(topic_term_prob.shape)     # (n_topics, n_terms)
print(topic_term_prob[0].sum())  # 1.0

(50,)
0.9999999
(50, 9774)
1.0


In [11]:
import pyLDAvis.gensim as gensimvis

prepared_data = gensimvis.prepare(lda_model, corpus, dictionary)

In [12]:
# pyLDAvis.display(prepared_data)

In [13]:
pyldavis_html_path = '../_output/pyLDAvis.html'
pyLDAvis.save_html(prepared_data, pyldavis_html_path)